<a href="https://www.cognitiveclass.ai"><img src = "https://cognitiveclass.ai/wp-content/themes/bdu3.0/static/images/cc-logo.png" align = left></a>

<br>
<br>
# Restoring model and prediction


--------------------
In this notebook we load the model that we built before, and use it for prediction of signals.  
__notice:__Make sure you have ran the `SETI_img_to-binary.ipynb` botebook, before running the following cells.


## Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import SETI
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn import metrics
# This is needed to display the images.
%matplotlib inline

## Read your test data
In the following cell, we read the test data

In [2]:
# working directory
dataset_name = 'SETI_ds_64x128'
train_dir = "tmp/SETI1_train/"
mydatafolder = "tmp/SETI1_data/"

# check point directory
chk_directory = train_dir + '/save1/' # it reads for the model built in the SETI_img_to-binary.ipynb
checkpoint_path = chk_directory + 'model.ckpt'

# Loading test set
dataset = SETI.read_data_sets(mydatafolder + dataset_name, one_hot=True, validation_size=0)
dataset.test.images.shape


Extracting tmp/SETI1_data/SETI_ds_64x128/train-images-idx3-ubyte.gz
Extracting tmp/SETI1_data/SETI_ds_64x128/train-labels-idx1-ubyte.gz
Extracting tmp/SETI1_data/SETI_ds_64x128/test-images-idx3-ubyte.gz
Extracting tmp/SETI1_data/SETI_ds_64x128/test-labels-idx1-ubyte.gz


(800, 8192)

## Restore the model
First, we load meta graph and restore weights

In [3]:
sess=tf.Session()    
filename = ".".join([tf.train.latest_checkpoint(chk_directory), "meta"])
saver = tf.train.import_meta_graph(filename)
saver.restore(sess,tf.train.latest_checkpoint(chk_directory))
print("Model restored.") 
print('Initialized')

INFO:tensorflow:Restoring parameters from tmp/SETI1_train//save1/model.ckpt-300
Model restored.
Initialized


## Reload the variables

In [4]:
detection_graph = tf.get_default_graph()
y_CNN = detection_graph.get_tensor_by_name("ReadoutLayer/softmax_linear:0")
x = detection_graph.get_tensor_by_name("x:0")
y_ = detection_graph.get_tensor_by_name("y_:0")
keep_prob = detection_graph.get_tensor_by_name("keep_prob:0")

## Prediction
Now we can simply use the test dataset to predict, we just have used 20 signals here for prediction.

In [6]:
x_batch, y_batch = dataset.test.next_batch(20 ,shuffle=True)
y_pred_lb = sess.run(tf.argmax(y_CNN,1), feed_dict={x: x_batch, y_: y_batch, keep_prob: 1.})
y_pred = sess.run(y_CNN, feed_dict={x: x_batch, y_: y_batch, keep_prob: 1.})

## Evaluation

In [7]:
y_truth_lb = np.argmax(y_batch,1) # ground truth
print metrics.classification_report(y_true= y_truth_lb, y_pred= y_pred_lb)
print metrics.confusion_matrix(y_true= y_truth_lb, y_pred= y_pred_lb)
print("Classification accuracy: %0.6f" % metrics.accuracy_score(y_true= y_truth_lb, y_pred= y_pred_lb) )
print("Log Loss: %0.6f" % metrics.log_loss(y_true= y_truth_lb, y_pred= y_pred) )

             precision    recall  f1-score   support

          0       1.00      0.80      0.89         5
          1       0.44      1.00      0.62         4
          2       0.67      0.29      0.40         7
          3       1.00      1.00      1.00         4

avg / total       0.77      0.70      0.69        20

[[4 0 1 0]
 [0 4 0 0]
 [0 5 2 0]
 [0 0 0 4]]
Classification accuracy: 0.700000
Log Loss: 0.791993


[Deep Learning with TensorFlow](http://cocl.us/SETI-NIMBIX-ML0102EN) is a free course in __cognitiveclass.ia__ where you can learn TensorFlow and Deep Learning togetherwe.

Also, running deep learning programs usually needs a high performance platform. PowerAI speeds up deep learning and AI. Built on IBM's Power Systems, PowerAI is a scalable software platform that accelerates deep learning and AI with blazing performance for individual users or enterprises. The PowerAI platform supports popular machine learning libraries and dependencies including Tensorflow, Caffe, Torch, and Theano. You can download a [free version of PowerAI](http://cocl.us/SETI-NIMBIX-PowerAI).

### Authors

<div class="teacher-image" style="    float: left;
    width: 115px;
    height: 115px;
    margin-right: 10px;
    margin-bottom: 10px;
    border: 1px solid #CCC;
    padding: 3px;
    border-radius: 3px;
    text-align: center;"><img class="alignnone wp-image-2258 " src="https://ibm.box.com/shared/static/tyd41rlrnmfrrk78jx521eb73fljwvv0.jpg" alt="Saeed Aghabozorgi" width="178" height="178" /></div>
#### Saeed Aghabozorgi

[Saeed Aghabozorgi](https://ca.linkedin.com/in/saeedaghabozorgi), PhD is Sr. Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

